In [2]:
!wget  -P /content/ "https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip"


--2019-08-01 11:46:51--  https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3380245855 (3.1G) [application/zip]
Saving to: ‘/content/MURA-v1.1.zip’

MURA-v1.1.zip       100%[===================>]   3.15G  19.6MB/s    in 2m 34s  

2019-08-01 11:49:26 (20.9 MB/s) - ‘/content/MURA-v1.1.zip’ saved [3380245855/3380245855]



In [3]:
!unzip /content/MURA-v1.1.zip


Output hidden; open in https://colab.research.google.com to view.

In [0]:
!rm -rf /content/MURA-v1.1/valid/XR_WRIST

In [0]:
import os
import numpy as np
import cv2
import keras.backend as K
import random
import tensorflow as tf

In [0]:
def load_path(root_path=r'/content/MURA-v1.1/train'):
    Path=[]
    labels=[]
    labelsDict={'ELBOW':0,'FINGER':1,'FOREARM':2,'HAND':3,'HUMERUS':4,'SHOULDER':5}
    for root,dirs,files in os.walk(root_path):
        for name in files:
            path_1=os.path.join(root,name)
            Path.append(path_1)         
    for i in range(len(Path)):
        path_1=Path[i].split('_')[1].split("/")[0]
        labelnumber=labelsDict.get(path_1)
        labels.append(labelnumber)
    labels=np.asarray(labels)
    return Path,labels

In [0]:
def load_image(Path=r'/content/MURA-v1.1/train',size=512):
    Images=[]
    for path in Path:
        try:
                image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image,(size,size))
                Images.append(image)
        except Exception as e:
            print(str(e))
    Images=np.asarray(Images).astype('float32')
    Images=Images.reshape(len(Path),size,size,1)
    mean = np.mean(Images)			#normalization
    std = np.std(Images)
    Images = (Images - mean) / std
    return Images

In [0]:
Path,labels=load_path(r'/content/MURA-v1.1/train')
#for i,j in zip(Path,labels):
#  print(i+" "+j)


In [0]:
Images=load_image(Path,64)
print("done")

done


In [0]:
#ob={}
#ob['TrainImages']=Images


In [0]:
    def train_model():
        model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu,input_shape=(64,64,1)),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(128,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=1),#no padding strides is one
        tf.keras.layers.Dropout(0.25),       
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(6,activation=tf.nn.softmax)
        ])
        model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    
        model.fit(Images,labels,epochs=20,verbose=2)
     
        return model

In [0]:
model=train_model();

Epoch 1/20
27056/27056 - 25s - loss: 0.7985 - acc: 0.7222
Epoch 2/20
27056/27056 - 25s - loss: 0.4603 - acc: 0.8437
Epoch 3/20
27056/27056 - 25s - loss: 0.3612 - acc: 0.8778
Epoch 4/20
27056/27056 - 25s - loss: 0.2969 - acc: 0.9019
Epoch 5/20
27056/27056 - 25s - loss: 0.2528 - acc: 0.9161
Epoch 6/20
27056/27056 - 25s - loss: 0.2208 - acc: 0.9242
Epoch 7/20
27056/27056 - 25s - loss: 0.2007 - acc: 0.9330
Epoch 8/20
27056/27056 - 25s - loss: 0.1748 - acc: 0.9418
Epoch 9/20
27056/27056 - 25s - loss: 0.1599 - acc: 0.9452
Epoch 10/20
27056/27056 - 25s - loss: 0.1514 - acc: 0.9490
Epoch 11/20
27056/27056 - 25s - loss: 0.1394 - acc: 0.9517
Epoch 12/20
27056/27056 - 25s - loss: 0.1270 - acc: 0.9569
Epoch 13/20
27056/27056 - 25s - loss: 0.1210 - acc: 0.9586
Epoch 14/20
27056/27056 - 25s - loss: 0.1169 - acc: 0.9600
Epoch 15/20
27056/27056 - 25s - loss: 0.1033 - acc: 0.9644
Epoch 16/20
27056/27056 - 25s - loss: 0.1058 - acc: 0.9641
Epoch 17/20
27056/27056 - 25s - loss: 0.1003 - acc: 0.9649
Epoch 

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 64, 64, 64)        640       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 63, 63, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 32, 32, 128)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 64)       

In [0]:
#Validation of test data
ValidPath,Validlabels=load_path(r'/content/MURA-v1.1/valid')
Validimages=load_image(ValidPath,64)

In [0]:
score = model.evaluate(Validimages, Validlabels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5253657154616479
Test accuracy: 0.9003152


In [0]:
model.save("mainmodel.h5")